# MovieLens 영화 장르 테이블 만들기

In [1]:
# MySQL Connector 모듈 및 pandas 모듈 import
import mysql.connector as mqc
import numpy as np
import pandas as pd

In [2]:
# MySQL 연결 객체 생성
cnx1 = mqc.connect(host='v.pdmnu.com', user='root', password='!Bigdata10', database='mlens')

# MySQL을 실질적으로 실행하는 cursor 객체 생성
cursor1 = cnx1.cursor()

In [3]:
# 실행할 쿼리문 입력
query = 'SELECT genres FROM mlens.movies;'

# cursor 객체를 통하여 쿼리 실행
cursor1.execute(query)

In [4]:
# 결과물을 저장할 리스트 선언
genres = []

# 결과물을 리스트에 저장
# 한 개의 컬럼만 조회하였지만 결과물이 튜플들의 리스트로 반환되기 때문에 0번째 인덱스의 값만 가져오도록 코드 작성
for _ in cursor1:
    genres.append(_[0])
genres

['Adventure|Animation|Children|Comedy|Fantasy',
 'Adventure|Children|Fantasy',
 'Comedy|Romance',
 'Comedy|Drama|Romance',
 'Comedy',
 'Action|Crime|Thriller',
 'Comedy|Romance',
 'Adventure|Children',
 'Action',
 'Action|Adventure|Thriller',
 'Comedy|Drama|Romance',
 'Comedy|Horror',
 'Adventure|Animation|Children',
 'Drama',
 'Action|Adventure|Romance',
 'Crime|Drama',
 'Drama|Romance',
 'Comedy',
 'Comedy',
 'Action|Comedy|Crime|Drama|Thriller',
 'Comedy|Crime|Thriller',
 'Crime|Drama|Horror|Mystery|Thriller',
 'Action|Crime|Thriller',
 'Drama|Sci-Fi',
 'Drama|Romance',
 'Drama',
 'Children|Drama',
 'Drama|Romance',
 'Adventure|Drama|Fantasy|Mystery|Sci-Fi',
 'Crime|Drama',
 'Drama',
 'Mystery|Sci-Fi|Thriller',
 'Adventure|Romance|IMAX',
 'Children|Drama',
 'Drama|Romance',
 'Crime|Drama',
 'Documentary|IMAX',
 'Children|Comedy',
 'Comedy|Romance',
 'Drama',
 'Drama|War',
 'Action|Crime|Drama',
 'Drama',
 'Action|Adventure|Fantasy',
 'Comedy|Drama|Thriller',
 'Drama|Romance',
 'Myst

In [5]:
# 결과물을 저장할 Series 객체 생성
genres_series = pd.Series(genres, name='Genres')
genres_series

0        Adventure|Animation|Children|Comedy|Fantasy
1                         Adventure|Children|Fantasy
2                                     Comedy|Romance
3                               Comedy|Drama|Romance
4                                             Comedy
                            ...                     
58093                             (no genres listed)
58094                                         Comedy
58095                                          Drama
58096                  Adventure|Drama|Horror|Sci-Fi
58097                             Action|Crime|Drama
Name: Genres, Length: 58098, dtype: object

In [7]:
# 가져온 내용을 ,(쉼표)를 기준으로 분할하여 DataFrame에 저장
genres_split = genres_series.str.split('|', expand=True)
genres_split

,0,1,2,3,4,5,6,7,8,9
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,Adventure,Children,Fantasy,None,None,None,None,None,None,None
2,Comedy,Romance,None,None,None,None,None,None,None,None
3,Comedy,Drama,Romance,None,None,None,None,None,None,None
4,Comedy,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
58093,(no genres listed),None,None,None,None,None,None,None,None,None
58094,Comedy,None,None,None,None,None,None,None,None,None
58095,Drama,None,None,None,None,None,None,None,None,None
58096,Adventure,Drama,Horror,Sci-Fi,None,None,None,None,None,None


In [11]:
genres_series = pd.Series()
for _ in range(len(genres_split.columns)):
    # 겹치는 이름을 삭제하기 위하여 모든 컬럼을 하나의 Series 객체로 합침
    genres_series = pd.concat([genres_series, genres_split[_]])
# unique() 메서드를 활용하여 중복값을 제거한 후 반환된 ndarray 객체를 다시 Series 객체로 생성
# 이후 값을 기준으로 재정렬
genres_series = pd.Series(genres_series.unique(), name='Genres')
genres_series.sort_values(inplace=True, ignore_index=True)
genres_series

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


0     (no genres listed)
1                 Action
2              Adventure
3              Animation
4               Children
5                 Comedy
6                  Crime
7            Documentary
8                  Drama
9                Fantasy
10             Film-Noir
11                Horror
12                  IMAX
13               Musical
14               Mystery
15               Romance
16                Sci-Fi
17              Thriller
18                   War
19               Western
20                  None
Name: Genres, dtype: object

In [12]:
# 정리된 Series 객체의 값만 추출하여 MySQL 쿼리에 대입할 수 있도록 튜플들의 리스트로 변환
value_params = list(zip(genres_series.T.values, ))
value_params

[('(no genres listed)',),
 ('Action',),
 ('Adventure',),
 ('Animation',),
 ('Children',),
 ('Comedy',),
 ('Crime',),
 ('Documentary',),
 ('Drama',),
 ('Fantasy',),
 ('Film-Noir',),
 ('Horror',),
 ('IMAX',),
 ('Musical',),
 ('Mystery',),
 ('Romance',),
 ('Sci-Fi',),
 ('Thriller',),
 ('War',),
 ('Western',),
 (None,)]

In [10]:
# MySQL 쿼리 실행
query = 'INSERT INTO genres (genre_name) VALUES (%s)'
try:
    cursor1.executemany(query, value_params)
except mqc.Error as err:
    print('오류가 발생하였습니다:', err)
finally:
    print(str(cursor1.rowcount) + '개의 행이 처리되었습니다.')
    cnx1.commit()

41개의 행이 처리되었습니다.


In [11]:
# MySQL 연결 닫기
cnx1.close()